In [ ]:
# ✅ Instalación de requerimientos (Python 3.9.6)
!pip install google-play-scraper sentence-transformers matplotlib seaborn pandas feedparser transformers nltk wordcloud pysentimiento

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import re
import feedparser

from google_play_scraper import reviews, app, Sort
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# Importar pysentimiento para el análisis de sentimiento
from pysentimiento import create_analyzer

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
    
sns.set(style='whitegrid')

In [3]:
# Función para obtener reviews desde Google Play Store
def get_playstore_reviews(app_id, lang='es', country='AR', days_back=90):
    result, _ = reviews(
        app_id,
        lang=lang,
        country=country,
        sort=Sort.NEWEST,  # Usamos el enum correcto
        count=2000
    )
    df = pd.DataFrame(result)
    df['date'] = pd.to_datetime(df['at'])
    df.rename(columns={'score': 'rating'}, inplace=True)
    cutoff = dt.datetime.now() - dt.timedelta(days=days_back)
    return df[df['date'] >= cutoff]

# Función para obtener reviews desde App Store vía RSS de iTunes
def get_itunes_reviews(app_store_id, days_back=90):
    # URL del RSS de reseñas de iTunes (formato JSON)
    url = f"https://itunes.apple.com/rss/customerreviews/id/{app_store_id}/json"
    feed = feedparser.parse(url)
    reviews_list = []
    # El primer entry suele ser metadata, por lo que se omite
    for entry in feed.entries[1:]:
        # Validar que la entrada tenga el campo de rating
        if 'im_rating' in entry:
            try:
                rating = int(entry['im_rating'])
            except:
                rating = None
        else:
            rating = None
        title = entry.get('title', '')
        # El contenido suele venir en 'content' o en 'summary'
        if 'content' in entry and len(entry.content) > 0:
            content = entry.content[0].value
        else:
            content = entry.get('summary', '')
        review_date = pd.to_datetime(entry.get('updated'))
        reviews_list.append({
            'date': review_date,
            'rating': rating,
            'title': title,
            'content': content
        })
    df = pd.DataFrame(reviews_list)
    if df.empty or "date" not in df.columns:
        return df
    cutoff = dt.datetime.now() - dt.timedelta(days=days_back)
    df = df[df['date'] >= cutoff]
    return df

In [ ]:
# Definir un diccionario con los IDs de cada app
apps = {
    "belo": {
         "play_store_id": "com.belo.android",
         "app_store_id": "1575614708"
    },
    "astropay": {
         "play_store_id": "com.astropaycard.android",
         "app_store_id": "1128476912"
    }
}

# Lista para almacenar los DataFrames de reviews
all_reviews_list = []

# Iterar sobre cada app y obtener reviews de ambas tiendas
for app_name, ids in apps.items():
    # Reviews desde Google Play
    play_reviews = get_playstore_reviews(ids["play_store_id"])
    if not play_reviews.empty:
        play_reviews["app"] = app_name
        play_reviews["store"] = "Play Store"
        all_reviews_list.append(play_reviews)
    
    # Reviews desde App Store (iTunes RSS)
    itunes_reviews = get_itunes_reviews(ids["app_store_id"])
    if not itunes_reviews.empty:
        itunes_reviews["app"] = app_name
        itunes_reviews["store"] = "App Store"
        all_reviews_list.append(itunes_reviews)

# Concatenar todas las reviews en un único DataFrame
all_reviews = pd.concat(all_reviews_list, ignore_index=True)

In [5]:
# Análisis de sentimiento con pysentimiento
analyzer = create_analyzer(task="sentiment", lang="es")

def analyze_sentiment(text):
    if not text or not isinstance(text, str):
        return None
    result = analyzer.predict(text)
    return result.output

all_reviews["sentiment"] = all_reviews["content"].apply(analyze_sentiment)

In [ ]:
# Cargar modelo de embeddings y pipeline de sentimiento
model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
sentiment_pipeline = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")

# Frases clave representativas para cada categoría
keywords = {
    "bug": ["error", "problemas", "bug", "no abre", "no funciona", "no me deja", "no puedo", "no anda", "no carga"],
    "feature": ["sería bueno", "me gustaría que tenga", "necesito que agreguen", "falta", "sumen", "es mejor", 
        "prefiero", "me gustaría que tenga", "sería genial si agregaran", "quisiera que se incluya", "necesito que ofrezcan"]
}

# Pre-codificar las keywords
keyword_embeddings = {k: model.encode(v, convert_to_tensor=True) for k, v in keywords.items()}

def classify_keywords_with_sentiment(text, threshold=0.6, bug_strict_threshold=0.7, feature_strict_threshold=0.75):
    """
    Evalúa la similitud entre la review y las frases clave, combinándola con el análisis de sentimiento.
    
    Se ajustan los umbrales en función del sentimiento obtenido:
    
    - Para bugs:
      * Si el sentimiento es NEG, se permite un umbral ligeramente menor (threshold - 0.1).
      * Si es POS, se exige un umbral mayor (bug_strict_threshold).
      * Si es NEU, se utiliza el umbral base (threshold).
      
    - Para feature requests:
      * Si el sentimiento es POS, se exige un umbral mayor (feature_strict_threshold).
      * Si es NEG o NEU, se utiliza el umbral base (threshold).
      
    Retorna una serie con dos valores booleanos: "is_bug" e "is_feature".
    """
    if not text or not isinstance(text, str):
        return pd.Series({"is_bug": False, "is_feature": False})
    
    # Obtener sentimiento (por ejemplo: "POS", "NEG", "NEU")
    sentiment_result = sentiment_pipeline(text)[0]
    sentiment_label = sentiment_result['label'].upper()
    
    # Calcular embedding del texto
    text_embedding = model.encode(text, convert_to_tensor=True)
    scores = {}
    for category, emb_list in keyword_embeddings.items():
        sim = util.cos_sim(text_embedding, emb_list)
        scores[category] = sim.max().item()
    
    # Ajuste del umbral para bugs según el sentimiento
    if sentiment_label == "NEG":
        bug_threshold = threshold - 0.1  # umbral más permisivo para bugs en reviews negativas
    elif sentiment_label == "POS":
        bug_threshold = bug_strict_threshold  # se exige mayor similitud si el sentimiento es positivo
    else:
        bug_threshold = threshold
    
    is_bug = scores["bug"] > bug_threshold
    
    # Ajuste del umbral para features según el sentimiento
    if sentiment_label == "POS":
        feature_threshold = feature_strict_threshold  # se exige mayor similitud para features en reviews positivas
    else:
        feature_threshold = threshold  # para NEU y NEG se usa el umbral base
    
    is_feature = scores["feature"] > feature_threshold
    
    return pd.Series({"is_bug": is_bug, "is_feature": is_feature})


tags = all_reviews["content"].apply(classify_keywords_with_sentiment)
all_reviews = pd.concat([all_reviews, tags], axis=1)

In [ ]:
def preprocess_text(text):
    """
    Preprocesa un texto: lo pasa a minúsculas, elimina caracteres no alfabéticos
    (conservando acentos y ñ) y reduce espacios múltiples.
    """
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres no alfabéticos (conservar letras con acento y ñ)
    text = re.sub(r"[^a-záéíóúñü\s]", "", text)
    # Eliminar espacios extra
    text = re.sub(r"\s+", " ", text).strip()
    return text

def extract_topics(reviews, n_topics=5, n_top_words=10, min_df=2, max_df=0.95):
    """
    Extrae los temas más recurrentes de un listado de reviews utilizando LDA.
    
    Parámetros:
      reviews (list o pd.Series): Lista o serie de textos de reviews.
      n_topics (int): Número de temas a extraer.
      n_top_words (int): Número de palabras clave que se mostrarán por tema.
      min_df (int): Frecuencia mínima para que una palabra se incluya.
      max_df (float): Fracción máxima de documentos en la que una palabra puede aparecer.
      
    Retorna:
      dict: Un diccionario donde las keys son nombres de temas y los valores son listas con las palabras clave.
    """
    # Preprocesar los reviews válidos
    preprocessed_reviews = [preprocess_text(text) for text in reviews if isinstance(text, str) and text.strip() != '']
    if not preprocessed_reviews:
        return {}
    
    # Usar las stop words en español de NLTK
    spanish_stopwords = stopwords.words('spanish')
    
    # Vectorizar el texto usando CountVectorizer con parámetros ajustados
    vectorizer = CountVectorizer(stop_words=spanish_stopwords, min_df=min_df, max_df=max_df)
    X = vectorizer.fit_transform(preprocessed_reviews)
    
    # Verificar que se hayan obtenido términos
    if X.shape[1] == 0:
        return {}
    
    # Aplicar LDA para extraer temas
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42, max_iter=10)
    lda.fit(X)
    
    feature_names = vectorizer.get_feature_names_out()
    topics = {}
    
    # Extraer las n_top_words de cada tema
    for topic_idx, topic in enumerate(lda.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        topics[f"Tema {topic_idx+1}"] = top_features
        
    return topics


for sentiment in all_reviews['sentiment'].unique():
    subset = all_reviews[all_reviews['sentiment'] == sentiment]['content']
    print(f"\nTópicos para reviews con sentimiento {sentiment}:")
    topics = extract_topics(subset, n_topics=5, n_top_words=10)
    for topic, words in topics.items():
        print(f"{topic}: {', '.join(words)}")

In [8]:
# Agregación semanal de las reviews
all_reviews['week'] = pd.to_datetime(all_reviews['date']).dt.to_period("W").apply(lambda r: r.start_time)

# Asegurarse de que las columnas 'sentiment', 'is_bug' e 'is_feature' existan
for col in ['sentiment', 'is_bug', 'is_feature']:
    if col not in all_reviews.columns:
         all_reviews[col] = None  

weekly_summary = all_reviews.groupby(['app', 'store', 'week']).agg(
    avg_rating=('rating', 'mean'),
    review_count=('rating', 'count'),
    positive_sentiment=('sentiment', lambda x: (x.str.contains("POS", case=False, na=False)).sum()),
    negative_sentiment=('sentiment', lambda x: (x.str.contains("NEG", case=False, na=False)).sum()),
    feature_requests=('is_feature', 'sum'),
    bug_reports=('is_bug', 'sum')
).reset_index()

In [ ]:
# Agrupar por app y sentimiento
sentiment_counts_app = all_reviews.groupby(['app', 'sentiment']).size().reset_index(name='Count')

plt.figure(figsize=(10, 6))
sns.barplot(x='sentiment', y='Count', hue='app', data=sentiment_counts_app, palette='viridis')
plt.title("Distribución de Sentimientos por App")
plt.xlabel("Sentimiento")
plt.ylabel("Cantidad de Reviews")
plt.show()

# Agrupar por semana, app y sentimiento
weekly_sentiment_app = all_reviews.groupby(['week', 'app', 'sentiment']).size().reset_index(name='Count')

plt.figure(figsize=(12, 6))
sns.lineplot(x='week', y='Count', hue='sentiment', style='app', data=weekly_sentiment_app, markers=True)
plt.title("Evolución Semanal de los Sentimientos por App")
plt.xlabel("Semana")
plt.ylabel("Cantidad de Reviews")
plt.xticks(rotation=45)
plt.legend(title="Sentimiento / App")
plt.show()

# Mapear el sentimiento a un score numérico
sentiment_map = {"POS": 1, "NEU": 0, "NEG": -1}
all_reviews["sentiment_score"] = all_reviews["sentiment"].map(sentiment_map)

# Visualización: Rating vs Sentiment Score
plt.figure(figsize=(10, 6))
# Boxplot para mostrar la distribución de 'sentiment_score' por rating
sns.boxplot(x="rating", y="sentiment_score", data=all_reviews, hue="app", showfliers=False)
# Stripplot para mostrar cada punto
sns.stripplot(x="rating", y="sentiment_score", data=all_reviews, hue="app", 
              dodge=True, alpha=0.5, color='black', jitter=True)

plt.title("Distribución de Sentiment Score por Rating (posible ironía)")
plt.xlabel("Rating")
plt.ylabel("Sentiment Score")
plt.legend(title="App", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Visualizar los tópicos por sentimiento usando WordCloud
for sentiment in all_reviews['sentiment'].unique():
    subset = all_reviews[all_reviews['sentiment'] == sentiment]['content']
    topics = extract_topics(subset, n_topics=5, n_top_words=10)
    
    # Combinar las palabras clave de todos los tópicos en una única cadena
    all_words = []
    for topic, words in topics.items():
         all_words.extend(words)
    text = " ".join(all_words)
    
    # Generar el WordCloud
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(text)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"WordCloud de tópicos para reviews con sentimiento {sentiment}")
    plt.show()

# Listado de Feature Requests
features_reviews = all_reviews[all_reviews['is_feature'] == True]
print("Listado de Feature Requests:")
display(features_reviews[['date', 'app', 'store', 'content', 'sentiment']].sort_values(by='date', ascending=False).reset_index(drop=True))

# Listado de Bug Reports para la app 'belo'
bugs_reviews = all_reviews[(all_reviews['is_bug'] == True) & (all_reviews['app'] == 'belo')]
print("Listado de Bug Reports:")
display(bugs_reviews[['date', 'app', 'store', 'content', 'sentiment']].sort_values(by='date', ascending=False).reset_index(drop=True))

In [9]:
# Guardar todas las reviews con análisis de sentimiento
all_reviews.to_csv("all_reviews.csv", index=False)

# Guardar el resumen semanal
weekly_summary.to_csv("weekly_summary.csv", index=False)

# Guardar feature requests
features_reviews.to_csv("features_reviews.csv", index=False)

# Guardar bug reports
bugs_reviews.to_csv("bugs_reviews.csv", index=False)